In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [23]:
import src.NearNear_v3 as nn
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
df = pd.read_csv('toy_data.csv')

In [57]:
df = df.iloc[:20,1:]

In [58]:
df.shape

(20, 5)

In [59]:
y = df.pop('sale_amount')
X = df
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)

### Look at shape and output of `compute_distances` function

In [60]:
lat_train = X_train['property_latitude'].values
lon_train = X_train['property_longitude'].values
lat_test = X_test['property_latitude'].values
lon_test = X_test['property_longitude'].values

In [61]:
dmat = nn.compute_distances(lat_train, lon_train, lat_test, lon_test)

In [65]:
dmat.shape

(7, 13)

#### Fit Model and predict on toy data

In [81]:
nn_model = nn.NearNear(lat='property_latitude',
                        lon='property_longitude')

In [82]:
nn_model.fit(X_train,y_train)

In [91]:
nn_model.predict(X_test)

array([ 9259259.22222222, 21717500.        ,  8836666.6       ,
       10316666.66666667,  3200000.        , 10976190.42857143,
        3250000.        ])

In [88]:
y_test

0     26600000.0
17     7000000.0
15     4786700.0
1      6005000.0
8      5850000.0
5     29500000.0
11     7677691.0
Name: sale_amount, dtype: float64